## **Build Custom Container with Custom prediction Routine and push it to Artifacts Registry**

In [90]:
from google.cloud.aiplatform.prediction import LocalModel 
from src.custom_sam_predictor import CustomSamPredictor 
import os
import logging

In [91]:
logging.basicConfig(level=logging.INFO)

In [92]:
USER_SRC_DIR = "src" # @param {type:"string"} source code directory name

""" Replace it with the region you will be working in on GCP """
REGION = "us-west1" # @param {type:"string"} region

""" Replace it with your Repository name """
REPOSITORY = "sam-container"  # @param {type:"string"} Artifacts registry repository name

""" Replace it with your PROJECT ID """
PROJECT_ID = "ml-ops-segment-anything" # @param {type:"string"} GCP project ID

IMAGE = "sam-cpr-container"  # @param {type:"string"} Custom Container name

LOCAL_MODEL_ARTIFACTS_DIR = "gs://segment-anything/ViT-B" # @param {type:"string"} Directory with model artifacts, Required for local deployment

In [93]:
""" Remove all unused containers """
!docker system prune -f

Deleted Images:
untagged: us-west1-docker.pkg.dev/ml-ops-segment-anything/sam-container/sam-cpr-container@sha256:38d5799a1b8fce57f668e8fb5f8bddfae2597b551839f5aa1d508e8f0e661019
deleted: sha256:5d559c64e82ea4cabf624451c6ac85205ab0e2b7c941c5bcd36107bce9303daa
deleted: sha256:9ebefc31a92562c3fcbe72209b9274d0fc8eec2bf8dd7cb50965a9bdf8f7865b

Total reclaimed space: 26.31kB


#### **Build custom container**

In [107]:
""" Build custom docker container """
local_model = LocalModel.build_cpr_model(
    USER_SRC_DIR,
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    predictor=CustomSamPredictor,
    # requirements_path=os.path.join(USER_SRC_DIR, "requirements.txt"),
    base_image="us-west1-docker.pkg.dev/ml-ops-segment-anything/sam-container/online-container"
    # base_image="pytorch/pytorch:1.13.1-cuda11.6-cudnn8-runtime" # Base image from docker hub
    # base_image="us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13.py310:latest"
    # base_image="us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13:latest"
)

INFO:google.cloud.aiplatform.docker_utils.build:Running command: docker build -t us-west1-docker.pkg.dev/ml-ops-segment-anything/sam-container/sam-cpr-container --rm -f- src
/opt/conda/lib/python3.10/subprocess.py:955: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/opt/conda/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
INFO:google.cloud.aiplatform.docker_utils.local_util:Sending build context to Docker daemon  31.28kB
INFO:google.cloud.aiplatform.docker_utils.local_util:

INFO:google.cloud.aiplatform.docker_utils.local_util:Step 1/13 : FROM us-west1-docker.pkg.dev/ml-ops-segment-anything/sam-container/online-container

INFO:google.cloud.aiplatform.docker_utils.local_util: ---> 8ec9167592f0

INFO:google.cloud.aiplatform.dock

In [108]:
local_model.get_serving_container_spec()

image_uri: "us-west1-docker.pkg.dev/ml-ops-segment-anything/sam-container/sam-cpr-container"
predict_route: "/predict"
health_route: "/health"

In [109]:
""" INPUT JSON file to test ENDPOINT """
INPUT_FILE_WITH_PROMPTS = "/home/jupyter/ml-ops-segment-anything/vertexAI_online_predictions/input_with_prompts.jsonl"
INPUT_FILE_WITHOUT_PROMPTS = "/home/jupyter/ml-ops-segment-anything/vertexAI_online_predictions/input_without_prompts.jsonl"

#### **Deploying Locally**

In [33]:
""" Deploying the model locally and simultaneously hitting the endpoint """

with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{LOCAL_MODEL_ARTIFACTS_DIR}",
    gpu_count=1
    
) as local_endpoint:
    print("predicting")
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE_WITH_PROMPTS,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

INFO:google.cloud.aiplatform.prediction.local_endpoint:Got the project id from the global config: ml-ops-segment-anything.


predicting


In [34]:
""" Printing model response """
print(predict_response.json())

{'prediction_type': 'Predicting without Prompts', 'ratio': 0.8737201365187713, 'polygon_vertices': {'mask_0': [[[309, 25], [310, 24], [314, 24], [315, 25], [316, 25], [319, 28], [319, 29], [321, 31], [321, 35], [320, 36], [320, 38], [319, 39], [319, 42], [320, 43], [320, 45], [322, 47], [322, 48], [323, 49], [324, 49], [325, 50], [326, 50], [327, 51], [331, 51], [332, 52], [336, 52], [337, 53], [339, 53], [340, 54], [342, 54], [343, 55], [345, 55], [346, 56], [348, 56], [349, 57], [350, 57], [351, 58], [352, 58], [353, 59], [355, 59], [356, 60], [357, 60], [358, 61], [359, 61], [360, 62], [361, 62], [362, 63], [363, 63], [365, 65], [366, 65], [368, 67], [369, 67], [371, 69], [372, 69], [376, 73], [377, 73], [398, 94], [398, 95], [401, 98], [401, 99], [404, 102], [404, 103], [406, 105], [406, 106], [408, 108], [408, 109], [409, 110], [409, 111], [410, 112], [410, 113], [411, 114], [411, 115], [412, 116], [412, 117], [413, 118], [413, 119], [414, 120], [414, 121], [415, 122], [415, 124],

In [110]:
""" Push the container to ARTIFACTS REGISTRY """
local_model.push_image()

/opt/conda/lib/python3.10/subprocess.py:955: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/opt/conda/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
INFO:google.cloud.aiplatform.docker_utils.local_util:Using default tag: latest

INFO:google.cloud.aiplatform.docker_utils.local_util:The push refers to repository [us-west1-docker.pkg.dev/ml-ops-segment-anything/sam-container/sam-cpr-container]

INFO:google.cloud.aiplatform.docker_utils.local_util:228f5776c5eb: Preparing

INFO:google.cloud.aiplatform.docker_utils.local_util:2563615b67ae: Preparing

INFO:google.cloud.aiplatform.docker_utils.local_util:bf4d6706efae: Preparing

INFO:google.cloud.aiplatform.docker_utils.local_util:db25cbeb7584: Preparing

INFO:google.cloud.aiplatform

In [111]:
from google.cloud import aiplatform

GCS_PATH_TO_MODEL_ARTIFACTS = "gs://segment-anything/ViT-B"
MODEL_DISPLAY_NAME = "wallace_test"

model = aiplatform.Model.upload(
    local_model=local_model,
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=GCS_PATH_TO_MODEL_ARTIFACTS,
    location=REGION
)

Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/633534855904/locations/us-west1/models/8628413100925648896/operations/1284696873684172800


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/633534855904/locations/us-west1/models/8628413100925648896/operations/1284696873684172800


Model created. Resource name: projects/633534855904/locations/us-west1/models/8628413100925648896@1


INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/633534855904/locations/us-west1/models/8628413100925648896@1


To use this Model in another session:


INFO:google.cloud.aiplatform.models:To use this Model in another session:


model = aiplatform.Model('projects/633534855904/locations/us-west1/models/8628413100925648896@1')


INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/633534855904/locations/us-west1/models/8628413100925648896@1')


### **Testing the ENDPOINT**

In [19]:
""" Making a curl call to make sure the endpoint is working 
Replace
    PROJECT_ID, ENDPOINT_ID and INPUT_DATA_FILE with the correct values after deployment 
"""
!curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-west1-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-west1/endpoints/${ENDPOINT_ID}:predict \
-d "@${INPUT_DATA_FILE}"

In [66]:
# import json
# with open("input_without_prompts.jsonl", "r") as f:
#     js = json.load(f)

In [100]:
# # Copyright 2020 Google LLC
# #
# # Licensed under the Apache License, Version 2.0 (the "License");
# # you may not use this file except in compliance with the License.
# # You may obtain a copy of the License at
# #
# #     https://www.apache.org/licenses/LICENSE-2.0
# #
# # Unless required by applicable law or agreed to in writing, software
# # distributed under the License is distributed on an "AS IS" BASIS,
# # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# # See the License for the specific language governing permissions and
# # limitations under the License.

# # [START aiplatform_predict_custom_trained_model_sample]
# from typing import Dict, List, Union

# from google.cloud import aiplatform
# from google.protobuf import json_format
# from google.protobuf.struct_pb2 import Value


# def predict_custom_trained_model_sample(
#     project: str,
#     endpoint_id: str,
#     instances: Union[Dict, List[Dict]],
#     location: str = "us-west1",
#     api_endpoint: str = "us-west1-aiplatform.googleapis.com",
# ):
#     """
#     `instances` can be either single instance of type dict or a list
#     of instances.
#     """
#     # The AI Platform services require regional API endpoints.
#     client_options = {"api_endpoint": api_endpoint}
#     # Initialize client that will be used to create and send requests.
#     # This client only needs to be created once, and can be reused for multiple requests.
#     client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
#     # The format of each instance should conform to the deployed model's prediction input schema.
#     instances = instances if isinstance(instances, list) else [instances]
#     instances = [
#         json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
#     ]
#     parameters_dict = {}
#     parameters = json_format.ParseDict(parameters_dict, Value())
#     endpoint = client.endpoint_path(
#         project=project, location=location, endpoint=endpoint_id
#     )
#     response = client.predict(
#         endpoint=endpoint, instances=instances, parameters=parameters
#     )
#     print("response")
#     print(" deployed_model_id:", response.deployed_model_id)
#     # The predictions are a google.protobuf.Value representation of the model's predictions.
#     predictions = response.predictions
#     for prediction in predictions:
#         print(" prediction:", dict(prediction))
#     return endpoint, response


# endpoint, response = predict_custom_trained_model_sample(
#     project="633534855904",
#     endpoint_id="1619791733946580992",
#     instances=js['instances']
# )
# # [END aiplatform_predict_custom_trained_model_sample]


response
 deployed_model_id: 6952599050520625152
 prediction: {'mask_6': [[[205.0, 146.0], [202.0, 149.0], [202.0, 153.0], [206.0, 157.0], [209.0, 157.0], [212.0, 154.0], [212.0, 153.0], [213.0, 152.0], [213.0, 150.0], [209.0, 146.0]]], 'mask_5': [[[310.0, 27.0], [305.0, 32.0], [305.0, 33.0], [304.0, 34.0], [304.0, 44.0], [303.0, 45.0], [303.0, 49.0], [304.0, 50.0], [304.0, 52.0], [307.0, 52.0], [312.0, 47.0], [312.0, 46.0], [314.0, 44.0], [314.0, 43.0], [315.0, 42.0], [315.0, 41.0], [316.0, 40.0], [316.0, 39.0], [317.0, 38.0], [317.0, 36.0], [318.0, 35.0], [318.0, 33.0], [317.0, 32.0], [317.0, 31.0], [315.0, 29.0], [315.0, 28.0], [314.0, 28.0], [313.0, 27.0]]], 'mask_4': [[[476.0, 189.0], [475.0, 190.0], [468.0, 190.0], [467.0, 191.0], [460.0, 191.0], [459.0, 192.0], [453.0, 192.0], [452.0, 193.0], [447.0, 193.0], [446.0, 194.0], [441.0, 194.0], [440.0, 195.0], [437.0, 195.0], [436.0, 196.0], [433.0, 196.0], [432.0, 197.0], [431.0, 197.0], [430.0, 198.0], [429.0, 198.0], [423.0, 204.0

ValueError: dictionary update sequence element #0 has length 1; 2 is required